In [29]:
import torch
import os


class Para:
    feature_column_start_name = 'ep_ratio_ttm' # 'VOL5'
    feature_column_end_name = 'BR' # 'AROON_DOWN'

    feature_num = 16 # 8

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_path = 'data/sk_space_1d_rate_20d_22-22_pre'
    model_path = 'models/model_sk_c2_s0.1_b16_lr0.001_d0.5_e20.pth'

    classification_n = 2

    month_num = len(os.listdir(data_path))

para = Para()
print(para.month_num)

1


In [30]:
import numpy as np
import torch
from my_utils.model_class import MLP
import pandas as pd

def insert_predict(df: pd.DataFrame, model_path, out_nums: int, column_name: str) -> pd.DataFrame:

    # 加载模型
    model = MLP(in_nums=para.feature_num, out_nums=out_nums, drop_p=0.5)
    model.load_state_dict(torch.load(model_path, map_location=para.device))
    model = model.to(para.device)

    # 运算
    model.eval()
    predict_tensor = model(torch.Tensor(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).to(device=para.device))

    # 插入 predict_tensor 列
    df.insert(loc=0, column='predict_tensor_'+column_name, value=np.nan)
    # 插入 predict_return_bin 列
    df.insert(loc=0, column='predict_return_bin_'+column_name, value=np.nan)

    df['predict_tensor'+'_'+column_name] = predict_tensor.to(device='cpu').detach().numpy().tolist()
    df['predict_return_bin'+'_'+column_name] = predict_tensor.to(device='cpu').detach().numpy().argmax(1)

    return df

In [31]:
df = pd.DataFrame()

for i_csv in range(para.month_num):
    df = pd.read_csv(para.data_path +'/' + str(i_csv) + '.csv', index_col='order_book_id')

df

,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
order_book_id,,,,,,,,,,,,,,,,,,,,,
600376.XSHG,MainBoard,RealEstate,0,2022-01-04,2.150414,0.885993,-0.904836,0.885993,0.509815,0.371793,...,0.584014,0.563670,0.736664,0.589693,0.984890,-2.406332,0.333570,-2.223795,0.653695,0.949237
600376.XSHG,MainBoard,RealEstate,1,2022-01-05,1.379074,0.014344,-0.044409,0.014344,0.642553,0.409583,...,0.260424,1.395753,1.165683,0.777085,1.253139,-1.980971,-0.217059,-1.976358,1.509365,1.592895
600376.XSHG,MainBoard,RealEstate,2,2022-01-06,1.793948,0.127999,-0.159133,0.127999,0.706257,0.457539,...,0.603361,1.395753,0.907695,0.847909,1.395975,-1.812021,-0.870438,-1.743586,1.467776,1.639934
600376.XSHG,MainBoard,RealEstate,3,2022-01-07,1.119903,-0.487725,0.471847,-0.487725,0.826973,0.529490,...,1.629346,1.395753,1.258799,0.960404,1.644152,-1.563709,-0.317418,-1.445352,1.725487,1.953677
600376.XSHG,MainBoard,RealEstate,4,2022-01-10,0.744371,-0.210673,0.185038,-0.210673,0.853349,0.594390,...,1.296049,1.395753,0.947061,1.039450,1.498971,-1.378603,-0.103479,-1.236502,0.736792,1.220539
600376.XSHG,MainBoard,RealEstate,5,2022-01-11,-0.262400,-0.974956,0.988103,-0.974956,0.968950,0.678474,...,1.537822,1.395753,1.389042,1.155654,1.594796,-1.084856,0.373692,-0.953711,1.452116,1.654475
600376.XSHG,MainBoard,RealEstate,6,2022-01-12,-0.676498,-1.344069,1.389636,-1.344069,1.092924,0.780233,...,1.544884,1.395753,1.525159,1.316973,1.578428,-0.818150,0.851443,-0.590917,1.362333,1.394781
600376.XSHG,MainBoard,RealEstate,7,2022-01-13,-0.985707,-1.551249,1.619083,-1.551249,1.196240,0.890387,...,1.564968,1.395753,1.534517,1.368277,1.602327,-0.643695,1.282435,-0.223983,1.231482,1.078255
600376.XSHG,MainBoard,RealEstate,8,2022-01-14,-0.357573,-1.239467,1.274912,-1.239467,1.196175,0.978491,...,1.223820,1.395753,1.166906,1.325482,1.625601,-0.468134,1.508347,0.080129,0.842428,0.758957


In [32]:
import torchmetrics
import time
import matplotlib.pyplot as plt
import torch.nn.functional as F

precision = torchmetrics.Precision(average='none', num_classes=para.classification_n)
accuracy = torchmetrics.Accuracy()

# 插入预测数据
df = insert_predict(df=df, model_path=para.model_path, out_nums=para.classification_n, column_name='2c')

# 综合多个预测数据
df.insert(loc=0, column='predict_return_bin', value=np.nan)
df.loc[df['predict_return_bin_2c'].isin([0,]), 'predict_return_bin'] = 0
df['predict_return_bin'].replace(to_replace=np.nan, value=1, inplace=True)
df['predict_return_bin'] = df['predict_return_bin'].astype('int64')

df.insert(loc=0, column='probability', value=np.nan)
for stock_id in range(len(df.index)):
    df.iloc[stock_id, 0] = F.softmax(torch.tensor(df.iloc[stock_id, df.columns.get_loc('predict_tensor_2c')]), dim=0).numpy()[0]

# 插入实际 return_bin 列
df.insert(loc=0, column='return_bin', value=np.nan)
df.loc[df['yield_rate'] > 0.0, 'return_bin'] = 0
df['return_bin'].replace(to_replace=np.nan, value=1, inplace=True)
df['return_bin'] = df['return_bin'].astype('int64')

# 计算精确率
df = df.sort_values(by='probability', ascending=False)

pred = torch.tensor(df['predict_return_bin']).type(torch.LongTensor)
target = torch.tensor(df['return_bin']).type(torch.LongTensor)

print(pred)
print(target)

print(precision(preds=pred, target=target))
print(accuracy(preds=pred, target=target))

print(df.loc[df['yield_rate']>0.0, 'predict_return_bin'].value_counts())
print(df['return_bin'].value_counts())

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1])
tensor([0.4667, 0.6250])
tensor(0.5484)
0    7
1    6
Name: predict_return_bin, dtype: int64
1    18
0    13
Name: return_bin, dtype: int64


In [33]:
from rqdatac import get_price
import rqdatac

rqdatac.init()
hs300_df = get_price(order_book_ids='000300.XSHG', start_date='2022-01-01', end_date='2022-03-22', fields='close')

c:\users\millet\miniconda3\envs\torch1.8-gpu\lib\site-packages\rqdatac\client.py:197: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


PermissionDenied: this license is only allowed to access through the education network

In [ ]:
hs300 = hs300_df.reset_index(level='order_book_id').drop(labels='order_book_id', axis=1)
hs300.rolling(5).mean().plot(grid=True)

## 测试模型预测

In [ ]:
import torch
from my_utils.model_class import MLP

model = MLP(in_nums=22, out_nums=2, drop_p=0.5)
model.load_state_dict(torch.load('models/model_c2_p[[0.0, 0.3], [0.7, 1.0]]_b32_lr0.001_d0.5_e500.pth', map_location=para.device))
model = model.to(para.device)

model.eval()
a = model(torch.Tensor([[0.5]*22]*4).to(device=para.device))
a